In [1]:
import os
import emstereolibpy as em
import pandas as pd
import csv
from ipywidgets import IntProgress
from IPython.display import display
import time
from emstereolibpy import CameraID

Check that we're up and running

In [2]:
CAM_FILES = '/home/marrabld/data/BRII_AIMS_CAL_FILES/AIMS_Cals'

In [3]:
em.version()

(1, 0)

Pick the file we wish to transform

In [17]:
from tkinter import Tk
from tkinter import filedialog

Tk().withdraw() # we don't want a full GUI, so keep the root window from appearing
filename = filedialog.askopenfilename()

In [18]:
df_track = pd.read_csv(filename)
df_track.head()

,FilenameLeft,FilenameRight,CamCalFile,Out_FileName,frame_left,id_left,color_left,ltrb_left,ltwh_left,center_left,frame_right,id_right,color_right,ltrb_right,ltwh_right,center_right
0,A000002_L.avi,A000002_R.avi,6429_RIG6,carcharhinidae_carcharhinus_amblyrhynchos-475.avi,73835,1,"(0, 255, 117)","(634.0, 665.0, 994.0, 842.0)","(634.0, 665.0, 360.0, 177.0)","(814.0, 753.5)",74155,1,"(0, 255, 117)","(585.0, 585.0, 991.0, 762.0)","(585.0, 585.0, 406.0, 177.0)","(788.0, 673.5)"
1,A000002_L.avi,A000002_R.avi,6429_RIG6,carcharhinidae_carcharhinus_amblyrhynchos-475.avi,73835,1,"(0, 255, 117)","(634.0, 665.0, 994.0, 842.0)","(634.0, 665.0, 360.0, 177.0)","(814.0, 753.5)",74155,2,"(234, 0, 255)","(475.0, 803.0, 763.0, 946.0)","(475.0, 803.0, 288.0, 143.0)","(619.0, 874.5)"
2,A000002_L.avi,A000002_R.avi,6429_RIG6,carcharhinidae_carcharhinus_amblyrhynchos-475.avi,73835,1,"(0, 255, 117)","(634.0, 665.0, 994.0, 842.0)","(634.0, 665.0, 360.0, 177.0)","(814.0, 753.5)",74155,3,"(126, 204, 0)","(254.0, 712.0, 598.0, 923.0)","(254.0, 712.0, 344.0, 211.0)","(426.0, 817.5)"
3,A000002_L.avi,A000002_R.avi,6429_RIG6,carcharhinidae_carcharhinus_amblyrhynchos-475.avi,73835,1,"(0, 255, 117)","(634.0, 665.0, 994.0, 842.0)","(634.0, 665.0, 360.0, 177.0)","(814.0, 753.5)",74155,4,"(0, 32, 204)","(336.0, 235.0, 734.0, 383.0)","(336.0, 235.0, 398.0, 148.0)","(535.0, 309.0)"
4,A000002_L.avi,A000002_R.avi,6429_RIG6,carcharhinidae_carcharhinus_amblyrhynchos-475.avi,73835,3,"(126, 204, 0)","(513.0, 269.0, 933.0, 448.0)","(513.0, 269.0, 420.0, 179.0)","(723.0, 358.5)",74155,1,"(0, 255, 117)","(585.0, 585.0, 991.0, 762.0)","(585.0, 585.0, 406.0, 177.0)","(788.0, 673.5)"


In [19]:
glob_tl_coords = []
glob_br_coords = []
x1 = 0
y1 = 1
x2 = 2
y2 = 3

glob_coords_dict = {
    'tlrb_xyz': [],
    'center_xyz': [],
    'tlrb_sd_xyz': [],
    'tlrb_rms': [],
    'center_sd_xyz': [],
    'center_rms': []
}

max_count = df_track.shape[0]
f = IntProgress(min=0, max=max_count) # instantiate the bar
display(f) # display the bar

for ii, row in df_track.iterrows():
    f.value += 1 # <-- update the progress bar

    print(f'Processing {ii} of {max_count}', end='\r', flush=True)

    # Each row needs it's own cam file. so we need to load the cam file for every row .. maybe, could optimise this better.
    result = em.load_camera_file(CameraID.left, os.path.join(CAM_FILES, df_track['CamCalFile'][ii] + '_L.Cam')) #'data/6429_RIG1_L.Cam'
    if result != 0:
        print(f"ERROR :: result = {result} for file {os.path.join(CAM_FILES, df_track['CamCalFile'][ii] + '_L.Cam')}")
        break
    result = em.load_camera_file(CameraID.right, os.path.join(CAM_FILES, df_track['CamCalFile'][ii] + '_R.Cam'))
    if result != 0:
        print(f'ERROR :: result = {result}')
        break

    # DELETE ME for debuggin cam file
    # result = em.load_camera_file(CameraID.left, 'data/6429_RIG1_L.Cam')
    # result = em.load_camera_file(CameraID.right, 'data/6429_RIG1_R.Cam')

    # <camera>_<pixel>
    left_coord = tuple(float(num) for num in df_track.ltrb_left[ii].replace('(', '').replace(')', '').split(', '))  # convert string  '(x1, y1, x2, y2)' to tuple of floats.
    right_coord = tuple(float(num) for num in df_track.ltrb_right[ii].replace('(', '').replace(')', '').split(', '))
    centre_left_coord = tuple(float(num) for num in df_track.center_left[ii].replace('(', '').replace(')', '').split(', '))
    centre_right_coord = tuple(float(num) for num in df_track.center_right[ii].replace('(', '').replace(')', '').split(', '))

    pt_left = em.Pt2D(left_coord[x1], left_coord[y1])   #top left of left BB
    pt_right = em.Pt2D(right_coord[x1], right_coord[y1]) # top left of right BB

    glob_tl_coords, glob_tl_sd, glob_tl_rms = em.intersect(pt_left, pt_right)

    pt_left = em.Pt2D(left_coord[x2], left_coord[y2])   #bottom right of left BB
    pt_right = em.Pt2D(right_coord[x2], right_coord[y2]) #Bottom right of right BB

    glob_br_coords, glob_br_sd, glob_br_rms = em.intersect(pt_left, pt_right)

    centre_pt_left = em.Pt2D(centre_left_coord[0], centre_left_coord[1])
    centre_pt_right = em.Pt2D(centre_right_coord[0], centre_right_coord[1])
    glob_centre_pt, glob_centre_sd, glob_centre_rms = em.intersect(centre_pt_left, centre_pt_right)

    glob_coords_dict['tlrb_xyz'].append(glob_tl_coords + glob_br_coords)
    glob_coords_dict['tlrb_sd_xyz'].append(glob_tl_sd + glob_br_sd)
    glob_coords_dict['tlrb_rms'].append((glob_tl_rms, glob_br_rms))

    glob_coords_dict['center_xyz'].append(glob_centre_pt)
    glob_coords_dict['center_sd_xyz'].append(glob_tl_sd + glob_br_sd)
    glob_coords_dict['center_rms'].append((glob_centre_rms, glob_centre_rms))


IntProgress(value=0, max=12108)

In [20]:
df_track['tlrb_xyz'] = glob_coords_dict['tlrb_xyz']
df_track['tlrb_sd_xyz'] = glob_coords_dict['tlrb_sd_xyz']
df_track['tlrb_rms'] = glob_coords_dict['tlrb_rms']

df_track['center_xyz'] = glob_coords_dict['center_xyz']
df_track['center_sd_xyz'] = glob_coords_dict['center_sd_xyz']
df_track['center_rms'] = glob_coords_dict['center_rms']

In [21]:
df_track.head()

,FilenameLeft,FilenameRight,CamCalFile,Out_FileName,frame_left,id_left,color_left,ltrb_left,ltwh_left,center_left,...,color_right,ltrb_right,ltwh_right,center_right,tlrb_xyz,tlrb_sd_xyz,tlrb_rms,center_xyz,center_sd_xyz,center_rms
0,A000002_L.avi,A000002_R.avi,6429_RIG6,carcharhinidae_carcharhinus_amblyrhynchos-475.avi,73835,1,"(0, 255, 117)","(634.0, 665.0, 994.0, 842.0)","(634.0, 665.0, 360.0, 177.0)","(814.0, 753.5)",...,"(0, 255, 117)","(585.0, 585.0, 991.0, 762.0)","(585.0, 585.0, 406.0, 177.0)","(788.0, 673.5)","(-283.04349083013534, -51.61415673742434, -128...","(0.5846944144886007, 0.4994951834723585, 1.828...","(5.1834034637281405, 7.6631447760465505)","(-148.37778358408337, -111.37360414751802, -13...","(0.5846944144886007, 0.4994951834723585, 1.828...","(4.888560015070182, 4.888560015070182)"
1,A000002_L.avi,A000002_R.avi,6429_RIG6,carcharhinidae_carcharhinus_amblyrhynchos-475.avi,73835,1,"(0, 255, 117)","(634.0, 665.0, 994.0, 842.0)","(634.0, 665.0, 360.0, 177.0)","(814.0, 753.5)",...,"(234, 0, 255)","(475.0, 803.0, 763.0, 946.0)","(475.0, 803.0, 288.0, 143.0)","(619.0, 874.5)","(-289.17558317787234, -113.08387221489383, -11...","(0.5313191333547348, 0.4688346906607986, 1.509...","(71.00437420165011, 57.03974312694329)","(-178.37557117093934, -159.25078461176835, -11...","(0.5313191333547348, 0.4688346906607986, 1.509...","(65.37610726565137, 65.37610726565137)"
2,A000002_L.avi,A000002_R.avi,6429_RIG6,carcharhinidae_carcharhinus_amblyrhynchos-475.avi,73835,1,"(0, 255, 117)","(634.0, 665.0, 994.0, 842.0)","(634.0, 665.0, 360.0, 177.0)","(814.0, 753.5)",...,"(126, 204, 0)","(254.0, 712.0, 598.0, 923.0)","(254.0, 712.0, 344.0, 211.0)","(426.0, 817.5)","(-304.02170620769084, -72.7926120735567, -938....","(0.43772871004221353, 0.3748838261624354, 1.02...","(37.7484171687221, 48.13767634608556)","(-209.19310861070022, -122.17386317579566, -96...","(0.43772871004221353, 0.3748838261624354, 1.02...","(44.35282578147962, 44.35282578147962)"
3,A000002_L.avi,A000002_R.avi,6429_RIG6,carcharhinidae_carcharhinus_amblyrhynchos-475.avi,73835,1,"(0, 255, 117)","(634.0, 665.0, 994.0, 842.0)","(634.0, 665.0, 360.0, 177.0)","(814.0, 753.5)",...,"(0, 32, 204)","(336.0, 235.0, 734.0, 383.0)","(336.0, 235.0, 398.0, 148.0)","(535.0, 309.0)","(-300.81179849441526, 44.64729147847362, -988....","(0.4598486129572666, 0.38938403491397106, 1.12...","(102.16660714644772, 113.40500692703965)","(-198.96690533938812, 9.265665766686304, -1037...","(0.4598486129572666, 0.38938403491397106, 1.12...","(107.4102848339837, 107.4102848339837)"
4,A000002_L.avi,A000002_R.avi,6429_RIG6,carcharhinidae_carcharhinus_amblyrhynchos-475.avi,73835,3,"(126, 204, 0)","(513.0, 269.0, 933.0, 448.0)","(513.0, 269.0, 420.0, 179.0)","(723.0, 358.5)",...,"(0, 255, 117)","(585.0, 585.0, 991.0, 762.0)","(585.0, 585.0, 406.0, 177.0)","(788.0, 673.5)","(-380.7279587506688, 104.7136597662103, -1477....","(0.7351811000712422, 0.5837740390319948, 2.370...","(138.8112025386879, 153.33490080459563)","(-214.941235707791, 41.21363686870612, -1532.6...","(0.7351811000712422, 0.5837740390319948, 2.370...","(149.80738331617465, 149.80738331617465)"


In [22]:
df_track.to_csv(filename.strip('.csv')+'-global.csv')